# Assignment 2 DSC 102 FA23

## Introduction

In this assignment we will conduct data engineering for the Amazon dataset. It is divided into 2 parts. The extracted features in Part 1 will be used for the Part 2 of assignment, where you train a model (or models) to predict user ratings for a product.

We will be using Apache Spark for this assignment. The default Spark API will be DataFrame, as it is now the recommended choice over the RDD API. That being said, please feel free to switch back to the RDD API if you see it as a better fit for the task. We provide you an option to request RDD format to start with. Also you can switch between DataFrame and RDD in your solution. 

Another newer API is Koalas, which is also avaliable. However, it has constraints and is not applicable to most tasks. Refer to the PA statement for detail.

### Set the following parameters

In [1]:
PID = 'U09809880' # your pid, for instance: 'a43223333'
INPUT_FORMAT = 'dataframe' # choose a format of your input data, valid options: 'dataframe', 'rdd', 'koalas'

In [2]:
# Boiler plates, do NOT modify
%load_ext autoreload
%autoreload 2
import os
import getpass
from pyspark.sql import SparkSession
from utilities import SEED
from utilities import PA2Test
from utilities import PA2Data
from utilities import data_cat
from pa2_main import PA2Executor
import time
if INPUT_FORMAT == 'dataframe':
    import pyspark.ml as M
    import pyspark.sql.functions as F
    import pyspark.sql.types as T
if INPUT_FORMAT == 'koalas':
    import databricks.koalas as ks
elif INPUT_FORMAT == 'rdd':
    import pyspark.mllib as M
    from pyspark.mllib.feature import Word2Vec
    from pyspark.mllib.linalg import Vectors
    from pyspark.mllib.linalg.distributed import RowMatrix

os.environ['PYSPARK_SUBMIT_ARGS'] = '--py-files utilities.py,assignment2.py \
--deploy-mode client \
pyspark-shell'

class args:
    review_filename = data_cat.review_filename
    product_filename = data_cat.product_filename
    product_processed_filename = data_cat.product_processed_filename
    ml_features_train_filename = data_cat.ml_features_train_filename
    ml_features_test_filename = data_cat.ml_features_test_filename
    output_root = '/home/{}/{}-pa2/test_results'.format(getpass.getuser(), PID)
    test_results_root = data_cat.test_results_root
    pid = PID

pa2 = PA2Executor(args, input_format=INPUT_FORMAT)
data_io = pa2.data_io
data_dict = pa2.data_dict
begin = time.time()


23/11/27 23:02:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/opt/bitnami/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Loading datasets ...Done


In [3]:
# Import your own dependencies


#-----------------------------

# Part 1: Feature Engineering

In [4]:
# Bring the part_1 datasets to memory and de-cache part_2 datasets. 
# Execute this once before you start working on this Part
data_dict, _ = data_io.cache_switch(data_dict, 'part_1')

# Task0: warm up 
This task is provided for you to get familiar with Spark API. We will use the dataframe API to demonstrate. Solution is given to you and this task won't be graded.

Refer to https://spark.apache.org/docs/latest/api/python/pyspark.sql.html for API guide.

The task is to implement the function below. Given the ```product_data``` table:
1. Take and print five rows.

1. Select only the ```asin``` column, then print five rows of it.

1. Select the row where ```asin = B00I8KEOTM``` and print it.

1. Count the total number of rows.

1. Calculate the mean ```price```.

1. You need to conduct the above operations, then extract some statistics out of the generated columns. You need to put the statistics in a python dictionary named ```res```. The description and schema of it are as follows:
    ```
    res
     | -- count_total: int -- count of total rows of the entire table after your operations
     | -- mean_price: float -- mean value of column price
    ```

In [5]:
def task_0(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    overall_column = 'overall'
    # Outputs:
    mean_rating_column = 'meanRating'
    count_rating_column = 'countRating'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------

    product_data.show(5)
    product_data[['asin']].show(5)
    product_data.where(F.col('asin') == 'B00I8KEOTM').show()
    count_rows = product_data.count()
    mean_price = product_data.select(F.avg(F.col('price'))).head()[0]
    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    # Calculate the values programmatically. Do not change the keys and do not
    # hard-code values in the dict. Your submission will be evaluated with
    # different inputs.
    # Modify the values of the following dictionary accordingly.
    res = {'count_total': None, 'mean_price': None}
    
    # Modify res:

    res['count_total'] = count_rows
    res['mean_price'] = mean_price

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    return res
    # -------------------------------------------------------------------------

In [6]:
if INPUT_FORMAT == 'dataframe':
    res = task_0(data_io, data_dict['product'])
    pa2.tests.test(res, 'task_0')

+----------+--------------------+--------------------+--------------------+-----+--------------------+
|      asin|           salesRank|          categories|               title|price|             related|
+----------+--------------------+--------------------+--------------------+-----+--------------------+
|B00I8HVV6E|{Home &amp; Kitch...|[[Home & Kitchen,...|Intelligent Desig...|27.99|{also_viewed -> [...|
|B00I8KEOTM|                null|[[Apps for Androi...|                null| null|{also_viewed -> [...|
|B00I8KCW4G|{Clothing -> 2233...|[[Clothing, Shoes...|eShakti Women's P...|41.95|{also_viewed -> [...|
|B00I8JKCQW|{Clothing -> 1405...|[[Clothing, Shoes...|Lady Slimming Mid...| null|{also_viewed -> [...|
|B00I8JKI8E|{Home &amp; Kitch...|[[Clothing, Shoes...|3 Tier Bangle Bra...|24.99|{also_viewed -> [...|
+----------+--------------------+--------------------+--------------------+-----+--------------------+
only showing top 5 rows

+----------+
|      asin|
+----------+
|B00I8HVV

# Task1

In [7]:
# %load -s task_1 assignment2.py
def task_1(data_io, review_data, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    overall_column = 'overall'
    # Outputs:
    mean_rating_column = 'meanRating'
    count_rating_column = 'countRating'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    ##spark = SparkSession.builder.appName("ProductRating").getOrCreate()
    
    result1 = (product_data.join(review_data, product_data["asin"]==review_data["asin"], "left_outer").groupBy(product_data["asin"]).agg(F.avg(F.col("overall")).alias("meanRating")))
    #result1.show()
    
    result2 = (product_data.join(review_data, product_data["asin"]==review_data["asin"], "left_outer").groupBy(product_data["asin"]).agg(F.when(F.count(F.col("overall")) > 0, F.count(F.col("overall"))).alias("countRating")))
    #result2.show()
    
    count_total = int(result1.count())
    #print("Total Rows (including null rows):", count_total)
    
    mean_meanRating = float(result1.agg(F.avg(F.col("meanRating"))).collect()[0][0])
    #print("mean value of meanRating: ", mean_meanRating)
    
    variance_meanRating = float(result1.agg(F.var_samp(F.col("meanRating"))).collect()[0][0])
    #print("Variance of meanRating:", variance_meanRating)
    
    nullNulls_meanRating = int(result1.filter(F.col("meanRating").isNull()).count())
    #print("Count of nulls in meanRating:", nullNulls_meanRating)
    
    mean_countRating = float(result2.agg(F.avg(F.col("countRating"))).collect()[0][0])
    #print("mean value of countRating: ", mean_countRating)   

    variance_countRating = float(result2.agg(F.var_samp(F.col("countRating"))).collect()[0][0])
    #print("Variance of countRating:", variance_countRating)
    
    nullNulls_countRating = int(result2.filter(F.col("countRating").isNull()).count())
    #print("Count of nulls in countRating:", nullNulls_countRating)   
    
    
    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    # Calculate the values programmaticly. Do not change the keys and do not
    # hard-code values in the dict. Your submission will be evaluated with
    # different inputs.
    # Modify the values of the following dictionary accordingly.
    res = {
        'count_total': None,
        'mean_meanRating': None,
        'variance_meanRating': None,
        'numNulls_meanRating': None,
        'mean_countRating': None,
        'variance_countRating': None,
        'numNulls_countRating': None
    }
    # Modify res:
    res['count_total']=count_total
    res['mean_meanRating']=mean_meanRating
    res['variance_meanRating']=variance_meanRating
    res['numNulls_meanRating']=nullNulls_meanRating
    res['mean_countRating']=mean_countRating
    res['variance_countRating']=variance_countRating
    res['numNulls_countRating']=nullNulls_countRating


    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_1')
    ##spark.stop()
    
    return res
    # -------------------------------------------------------------------------


In [8]:
res = task_1(data_io, data_dict['review'], data_dict['product'])
pa2.tests.test(res, 'task_1')

tests for task_1 --------------------------------------------------------------
Test 1/7 : count_total ... Pass
Test 2/7 : mean_countRating ... Pass
Test 3/7 : mean_meanRating ... Pass
Test 4/7 : numNulls_countRating ... Pass
Test 5/7 : numNulls_meanRating ... Pass
Test 6/7 : variance_countRating ... Pass
Test 7/7 : variance_meanRating ... Pass
7/7 passed
-------------------------------------------------------------------------------


True


# Task 2

In [9]:
# %load -s task_2 assignment2.py
def task_2(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    salesRank_column = 'salesRank'
    categories_column = 'categories'
    asin_column = 'asin'
    # Outputs:
    category_column = 'category'
    bestSalesCategory_column = 'bestSalesCategory'
    bestSalesRank_column = 'bestSalesRank'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    ##spark = SparkSession.builder.appName("Categories_SalesRank").getOrCreate()
    
    category_withoutBlank=product_data.withColumn("categories", F.when(F.col("categories").getItem(0).getItem(0) == "", None).otherwise(F.col("categories")))

    category_column = (category_withoutBlank.withColumn("category", F.when(F.col("categories").isNotNull() & F.col("categories").getItem(0).getItem(0).isNotNull() & F.col("categories").getItem(0).isNotNull(), F.col("categories").getItem(0).getItem(0))).select("category")) 
    #category_column.show()
    
    category_column_check=category_column.distinct()
    #category_column_check.show(100,truncate=False)
    
    test=category_column.filter(F.col("category").isNotNull()).select("category").distinct()
    #test.show(100,truncate=False)
    
    bestSalesCategory_column=(product_data
    .withColumn("bestSalesCategory", F.when(F.map_keys(F.col("salesRank")).isNotNull()&F.map_keys(F.col("salesRank")).isNotNull()&F.map_keys(F.col("salesRank"))[0].isNotNull(), F.map_keys(F.col("salesRank"))[0]))
    .select('bestSalesCategory'))
    #bestSalesCategory_column.show()
    

    bestSalesRank_column=(product_data
    .withColumn('bestSalesRank', F.when(F.map_keys(F.col("salesRank")).isNotNull()&F.map_values(F.col("salesRank"))[0].isNotNull(), F.map_values(F.col("salesRank"))[0]))
    .select('bestSalesRank'))
    #bestSalesRank_column.show()
    
    count_total=int(category_column.count())
    
    mean_bestSalesRank=float(bestSalesRank_column.agg(F.avg(F.col("bestSalesRank"))).collect()[0][0])
    
    variance_bestSalesRank=float(bestSalesRank_column.agg(F.var_samp(F.col("bestSalesRank"))).collect()[0][0])
    
    numNulls_category = int(category_column.filter(F.col("category").isNull()).count())
    
    countDistinct_category = int(category_column.filter(F.col("category").isNotNull()).select("category").distinct().count())
    
    numNulls_bestSalesCategory=int(bestSalesCategory_column.filter(F.col("bestSalesCategory").isNull()).count())
    
    countDistinct_bestSalesCategory=int(bestSalesCategory_column.filter(F.col("bestSalesCategory").isNotNull()).select("bestSalesCategory").distinct().count())


    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'mean_bestSalesRank': None,
        'variance_bestSalesRank': None,
        'numNulls_category': None,
        'countDistinct_category': None,
        'numNulls_bestSalesCategory': None,
        'countDistinct_bestSalesCategory': None
    }
    # Modify res:
    res['count_total']=count_total
    res['mean_bestSalesRank']=mean_bestSalesRank
    res['variance_bestSalesRank']=variance_bestSalesRank
    res['numNulls_category']=numNulls_category
    res['countDistinct_category']=countDistinct_category
    res['numNulls_bestSalesCategory']=numNulls_bestSalesCategory
    res['countDistinct_bestSalesCategory']=countDistinct_bestSalesCategory




    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_2')
    ##spark.stop()
    
    return res
    # -------------------------------------------------------------------------


In [10]:
res = task_2(data_io, data_dict['product'])
pa2.tests.test(res, 'task_2')

tests for task_2 --------------------------------------------------------------
Test 1/7 : countDistinct_bestSalesCategory ... Pass
Test 2/7 : countDistinct_category ... Pass
Test 3/7 : count_total ... Pass
Test 4/7 : mean_bestSalesRank ... Pass
Test 5/7 : numNulls_bestSalesCategory ... Pass
Test 6/7 : numNulls_category ... Pass
Test 7/7 : variance_bestSalesRank ... Pass
7/7 passed
-------------------------------------------------------------------------------


True

# Task 3





In [11]:
# %load -s task_3 assignment2.py
def task_3(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    price_column = 'price'
    attribute = 'also_viewed'
    related_column = 'related'
    # Outputs:
    meanPriceAlsoViewed_column = 'meanPriceAlsoViewed'
    countAlsoViewed_column = 'countAlsoViewed'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    exploded_df = product_data.select("asin", F.explode("related.also_viewed").alias("also_viewed"))
    #exploded_df.show()
    
    price_df=product_data.select("asin","price")
    #price_df.show()
    
    meanPriceAlsoViewed_column=(exploded_df.join(price_df, exploded_df["also_viewed"]==price_df["asin"],'left_outer').groupBy(exploded_df["asin"]).agg(F.avg(F.col("price")).alias("meanPriceAlsoViewed_column")))
    #meanPriceAlsoViewed_column.show()
    
    countAlsoViewed_column=(exploded_df.groupBy(exploded_df["asin"]).agg(F.count(F.col("also_viewed")).alias('countAlsoViewed')))
    #countAlsoViewed_column.show()
    
    
    
    product_data_output = product_data.join(meanPriceAlsoViewed_column, [asin_column], how='left')
    product_data_output = product_data_output.join(countAlsoViewed_column, [asin_column], how='left')
    numNulls_meanPriceAlsoViewed=int(product_data_output.filter(F.col("meanPriceAlsoViewed_column").isNull()).count())
    #print(numNulls_meanPriceAlsoViewed)
    numNulls_countAlsoViewed=int(product_data_output.filter(F.col('countAlsoViewed').isNull()).count())
    #print(numNulls_countAlsoViewed)
    
    count_total=int(product_data_output.count())
    #print(count_total)

    
    mean_meanPriceAlsoViewed=float(meanPriceAlsoViewed_column.agg(F.avg(F.col("meanPriceAlsoViewed_column"))).collect()[0][0])
    #print(mean_meanPriceAlsoViewed)
    
    variance_meanPriceAlsoViewed=float(meanPriceAlsoViewed_column.agg(F.var_samp(F.col("meanPriceAlsoViewed_column"))).collect()[0][0])
    #print(variance_meanPriceAlsoViewed)
    
    mean_countAlsoViewed=float(countAlsoViewed_column.agg(F.avg(F.col('countAlsoViewed'))).collect()[0][0])
    #print(mean_countAlsoViewed)
    
    variance_countAlsoViewed=float(countAlsoViewed_column.agg(F.var_samp(F.col('countAlsoViewed'))).collect()[0][0])
    #print(variance_countAlsoViewed)
    
    
    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'mean_meanPriceAlsoViewed': None,
        'variance_meanPriceAlsoViewed': None,
        'numNulls_meanPriceAlsoViewed': None,
        'mean_countAlsoViewed': None,
        'variance_countAlsoViewed': None,
        'numNulls_countAlsoViewed': None
    }
    # Modify res:
    res['count_total']=count_total
    res['mean_meanPriceAlsoViewed']=mean_meanPriceAlsoViewed
    res['variance_meanPriceAlsoViewed']=variance_meanPriceAlsoViewed
    res['numNulls_meanPriceAlsoViewed']=numNulls_meanPriceAlsoViewed
    res['mean_countAlsoViewed']=mean_countAlsoViewed
    res['variance_countAlsoViewed']=variance_countAlsoViewed
    res['numNulls_countAlsoViewed']=numNulls_countAlsoViewed
    



    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_3')

    return res
    # -------------------------------------------------------------------------


In [12]:
res = task_3(data_io, data_dict['product'])
pa2.tests.test(res, 'task_3')

tests for task_3 --------------------------------------------------------------
Test 1/7 : count_total ... Pass
Test 2/7 : mean_countAlsoViewed ... Pass
Test 3/7 : mean_meanPriceAlsoViewed ... Pass
Test 4/7 : numNulls_countAlsoViewed ... Pass
Test 5/7 : numNulls_meanPriceAlsoViewed ... Pass
Test 6/7 : variance_countAlsoViewed ... Pass
Test 7/7 : variance_meanPriceAlsoViewed ... Pass
7/7 passed
-------------------------------------------------------------------------------


True

# Task 4

In [13]:
# %load -s task_4 assignment2.py
def task_4(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    price_column = 'price'
    title_column = 'title'
    # Outputs:
    meanImputedPrice_column = 'meanImputedPrice'
    medianImputedPrice_column = 'medianImputedPrice'
    unknownImputedTitle_column = 'unknownImputedTitle'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    price_column = product_data.select(F.col("price").cast("float"))
    #price_column.show()
    
    mean_price = price_column.select(F.mean("price")).collect()[0][0]
    
    meanImputedPrice_column = price_column.withColumn("meanImputedPrice", F.when(F.col("price").isNull(), mean_price).otherwise(F.col("price")))
    #meanImputedPrice_column.show()
    
    median_price = price_column.approxQuantile("price", [0.5],0.05)[0]
    
    medianImputedPrice_column=price_column.withColumn("medianImputedPrice", F.when(F.col("price").isNull(), median_price).otherwise(F.col("price")))
    #medianImputedPrice_column.show()
    
    title_column=product_data.select('title')
    #title_column.show()
    
    unknownImputedTitle_column = title_column.withColumn("unknownImputedTitle", F.when((F.col("title").isNull()) | (F.col("title") == ""), "unknown").otherwise(F.col("title")))
    #unknownImputedTitle_column.show()

    
    count_total=int(meanImputedPrice_column.count())
    #print(count_total)
    
    mean_meanImputedPrice=float(meanImputedPrice_column.agg(F.avg(F.col("meanImputedPrice"))).collect()[0][0])
    #print(mean_meanImputedPrice)

    variance_meanImputedPrice=float(meanImputedPrice_column.agg(F.var_samp(F.col("meanImputedPrice"))).collect()[0][0])
    #print(variance_meanImputedPrice)
    
    numNulls_meanImputedPrice=int(meanImputedPrice_column.filter(F.col("meanImputedPrice").isNull()).count())
    #print(numNulls_meanImputedPrice)
    
    mean_medianImputedPrice=float(medianImputedPrice_column.agg(F.avg(F.col("medianImputedPrice"))).collect()[0][0])
    #print(mean_medianImputedPrice)
    
    variance_medianImputedPrice=float(medianImputedPrice_column.agg(F.var_samp(F.col("medianImputedPrice"))).collect()[0][0])
    #print(variance_medianImputedPrice)
    
    numNulls_medianImputedPrice=int(medianImputedPrice_column.filter(F.col("medianImputedPrice").isNull()).count())
    #print(numNulls_medianImputedPrice)
    
    numUnknowns_unknownImputedTitle=unknownImputedTitle_column.filter(F.col("unknownImputedTitle") == "unknown").count()
    #print(numUnknowns_unknownImputedTitle)

    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'mean_meanImputedPrice': None,
        'variance_meanImputedPrice': None,
        'numNulls_meanImputedPrice': None,
        'mean_medianImputedPrice': None,
        'variance_medianImputedPrice': None,
        'numNulls_medianImputedPrice': None,
        'numUnknowns_unknownImputedTitle': None
    }
    # Modify res:
    res['count_total']=count_total
    res['mean_meanImputedPrice']=mean_meanImputedPrice
    res['variance_meanImputedPrice']=variance_meanImputedPrice
    res['numNulls_meanImputedPrice']=numNulls_meanImputedPrice
    res['mean_medianImputedPrice']=mean_medianImputedPrice
    res['variance_medianImputedPrice']=variance_medianImputedPrice
    res['numNulls_medianImputedPrice']=numNulls_medianImputedPrice
    res['numUnknowns_unknownImputedTitle']=numUnknowns_unknownImputedTitle


    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_4')
    return res
    # -------------------------------------------------------------------------


In [14]:
res = task_4(data_io, data_dict['product'])
pa2.tests.test(res, 'task_4')

tests for task_4 --------------------------------------------------------------
Test 1/8 : count_total ... Pass
Test 2/8 : mean_meanImputedPrice ... Pass
Test 3/8 : mean_medianImputedPrice ... Pass
Test 4/8 : numNulls_meanImputedPrice ... Pass
Test 5/8 : numNulls_medianImputedPrice ... Pass
Test 6/8 : numUnknowns_unknownImputedTitle ... Pass
Test 7/8 : variance_meanImputedPrice ... Pass
Test 8/8 : variance_medianImputedPrice ... Pass
8/8 passed
-------------------------------------------------------------------------------


True

# Task 5

In [15]:
# %load -s task_5 assignment2.py
def task_5(data_io, product_processed_data, word_0, word_1, word_2):
    # -----------------------------Column names--------------------------------
    # Inputs:
    title_column = 'title'
    # Outputs:
    titleArray_column = 'titleArray'
    titleVector_column = 'titleVector'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------

    title_column=product_processed_data.select('title')
    
    titleArray_column=title_column.withColumn("titleArray", F.split(F.lower(F.col("title")), " "))
    #titleArray_column.show()
    
    titleArray_column=titleArray_column.select("titleArray")
    #titleArray_column.show()
    
    product_processed_data_output=titleArray_column
    #product_processed_data_output.show()
    
    word2Vec = M.feature.Word2Vec(
    vectorSize=16,
    minCount=100,
    seed=102,
    numPartitions=4,
    inputCol="titleArray",
    outputCol="word2vec")
    
    model = word2Vec.fit(titleArray_column)

    #model.save("word2vec_model")
    
    #model = M.feature.Word2VecModel.load("word2vec_model")
        
    count_total=product_processed_data_output.count()
    #print(count_total)
    
    size_vocabulary = model.getVectors().count()
    #print(size_vocabulary)
    
    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'size_vocabulary': None,
        'word_0_synonyms': [(None, None), ],
        'word_1_synonyms': [(None, None), ],
        'word_2_synonyms': [(None, None), ]
    }
    # Modify res:
    res['count_total'] = product_processed_data_output.count()
    res['size_vocabulary'] = model.getVectors().count()
    for name, word in zip(
        ['word_0_synonyms', 'word_1_synonyms', 'word_2_synonyms'],
        [word_0, word_1, word_2]
    ):
        res[name] = model.findSynonymsArray(word, 10)
    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_5')
    return res
    # -------------------------------------------------------------------------


In [16]:
res = task_5(data_io, data_dict['product_processed'], 'piano', 'rice', 'laptop')
pa2.tests.test(res, 'task_5')

23/11/27 23:14:33 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/27 23:14:33 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
tests for task_5 --------------------------------------------------------------
Test 1/8 : count_total ... Pass
Test 2/8 : size_vocabulary ... Pass
Test 3/8 : word_0_synonyms-length ... Pass
Test 4/8 : word_0_synonyms-correctness ... Pass
Test 5/8 : word_1_synonyms-length ... Pass
Test 6/8 : word_1_synonyms-correctness ... Pass
Test 7/8 : word_2_synonyms-length ... Pass
Test 8/8 : word_2_synonyms-correctness ... Pass
5/5 passed
-------------------------------------------------------------------------------


True

# Task 6

In [17]:
# %load -s task_6 assignment2.py
def task_6(data_io, product_processed_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    category_column = 'category'
    # Outputs:
    categoryIndex_column = 'categoryIndex'
    categoryOneHot_column = 'categoryOneHot'
    categoryPCA_column = 'categoryPCA'
    # -------------------------------------------------------------------------    

    # ---------------------- Your implementation begins------------------------
    indexer = M.feature.StringIndexer(inputCol="category", outputCol="categoryIndex")
    
    encoder = M.feature.OneHotEncoder(inputCol="categoryIndex", outputCol="categoryOneHot", dropLast=False)
    
    pca = M.feature.PCA(k=15, inputCol="categoryOneHot", outputCol="categoryPCA")
    
    pipeline = M.Pipeline(stages=[indexer, encoder, pca])
    
    category_column=product_processed_data.select('category')
    model = pipeline.fit(category_column)
    df_transformed = model.transform(category_column)
    #df_transformed.show()
    
    count_total=int(df_transformed.count())
    #print(count_total)
    
    #summarizer = M.stat.Summarizer.metrics("mean")
    
    meanVector_categoryOneHot=df_transformed.select(M.stat.Summarizer.mean(df_transformed.categoryOneHot))
    #meanVector_categoryOneHot.show()
    #print(type(meanVector_categoryOneHot))
    meanVector_categoryOneHot=meanVector_categoryOneHot.collect()
    ##meanVector_categoryOneHot=meanVector_categoryOneHot.toArray().tolist()
    #print(meanVector_categoryOneHot)
    #print(type(meanVector_categoryOneHot))
    
    list_meanVector_categoryOneHot=[]
    for i in meanVector_categoryOneHot:
        #print(i)
        for j in i:
            #print(j)
            for z in j:
                #print(z)
                list_meanVector_categoryOneHot.append(float(z))
    #print(list_meanVector_categoryOneHot)
    #meanVector_categoryOneHot=[0.2513, 0.1523, 0.0562, 0.0525, 0.0521, 0.0463, 0.0368, 0.0356, 0.0351, 0.0287, 0.0286, 0.0285, 0.0278, 0.0275, 0.0207, 0.0182, 0.0142, 0.0124, 0.0117, 0.0115, 0.0076, 0.0073, 0.0065, 0.0052, 0.005, 0.0032, 0.0026, 0.0019, 0.0012, 0.0011, 0.0008, 0.0008, 0.0007, 0.0007, 0.0006, 0.0005, 0.0005, 0.0005, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0003, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    
    #common_list = [row.mean(categoryOneHot).toArray().tolist() for row in meanVector_categoryOneHot]
    #print(common_list)
    
    meanVector_categoryPCA=df_transformed.select(M.stat.Summarizer.mean(df_transformed.categoryPCA))
    #meanVector_categoryPCA.show()
    meanVector_categoryPCA=meanVector_categoryPCA.collect()
    #meanVector_categoryPCA=meanVector_categoryPCA.tolist()
    #print(meanVector_categoryPCA)
    #print(type(meanVector_categoryPCA))
    list_meanVector_categoryPCA=[]
    for i in meanVector_categoryPCA:
        #print(i)
        for j in i:
            #print(j)
            for z in j:
                #print(z)
                list_meanVector_categoryPCA.append(float(z))  
    #print(list_meanVector_categoryPCA)
    #meanVector_categoryPCA=[-0.15, -0.1752, 0.0165, -0.0027, 0.0334, -0.0537, 0.0099, -0.0046, 0.04, -0.0003, -0.0019, 0.0061, -0.0037, 0.0372, -0.0305]
    
    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'meanVector_categoryOneHot': [None, ],
        'meanVector_categoryPCA': [None, ]
    }
    # Modify res:
    res['count_total']=count_total
    res['meanVector_categoryOneHot']=list_meanVector_categoryOneHot
    res['meanVector_categoryPCA']=list_meanVector_categoryPCA



    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_6')
    return res
    # -------------------------------------------------------------------------


In [18]:
res = task_6(data_io, data_dict['product_processed'])
pa2.tests.test(res, 'task_6')

23/11/27 23:14:49 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/11/27 23:14:49 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


tests for task_6 --------------------------------------------------------------
Test 1/9 : count_total ... Pass
Test 2/9 : meanVector_categoryOneHot-length ... Pass
Test 3/9 : meanVector_categoryOneHot-sum ... Pass
Test 4/9 : meanVector_categoryOneHot-mean ... Pass
Test 5/9 : meanVector_categoryOneHot-variance ... Pass
Test 6/9 : meanVector_categoryPCA-length ... Pass
Test 7/9 : meanVector_categoryPCA-sum ... Pass
Test 8/9 : meanVector_categoryPCA-mean ... Pass
Test 9/9 : meanVector_categoryPCA-variance ... Pass
9/9 passed
-------------------------------------------------------------------------------


True

In [19]:
print ("End to end time: {}".format(time.time()-begin))

End to end time: 750.132972240448


# Part 2: Model Selection

In [20]:
# Bring the part_2 datasets to memory and de-cache part_1 datasets.
# Execute this once before you start working on this Part
data_dict, _ = data_io.cache_switch(data_dict, 'part_2')

# Task 7

In [21]:
def task_7(data_io, train_data, test_data):
    
    # ---------------------- Your implementation begins------------------------
    features = train_data.columns[:-1] 
    target = "overall"
    
    dt_model = M.regression.DecisionTreeRegressor(featuresCol="features", labelCol=target, maxDepth=5)
    dt_model = dt_model.fit(train_data)
    
    predictions = dt_model.transform(test_data)

    evaluator = M.evaluation.RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    rmse = float(evaluator.evaluate(predictions))
    #print(rmse)
    
    
    # -------------------------------------------------------------------------
    
    
    # ---------------------- Put results in res dict --------------------------
    res = {
        'test_rmse': None
    }
    # Modify res:
    res['test_rmse']=rmse

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_7')
    return res
    # -------------------------------------------------------------------------

In [22]:
res = task_7(data_io, data_dict['ml_features_train'], data_dict['ml_features_test'])
pa2.tests.test(res, 'task_7')

tests for task_7 --------------------------------------------------------------
Test 1/1 : test_rmse ... Pass
1/1 passed
-------------------------------------------------------------------------------


True

# Task 8

In [23]:
def task_8(data_io, train_data, test_data):
    
    # ---------------------- Your implementation begins------------------------
    new_train_data, val_data = train_data.randomSplit([0.75, 0.25], seed=42)
    
    features = train_data.columns[:-1]
    target = "overall"
    
    dt_model =  M.regression.DecisionTreeRegressor(featuresCol="features", labelCol=target, maxDepth=5)
    dt_model = dt_model.fit(new_train_data)
    predictions = dt_model.transform(val_data)
    evaluator = M.evaluation.RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    valid_rmse_depth_5 = float(evaluator.evaluate(predictions))
    #print(valid_rmse_depth_5)
    
    dt_model =  M.regression.DecisionTreeRegressor(featuresCol="features", labelCol=target, maxDepth=7)
    dt_model = dt_model.fit(new_train_data)
    predictions = dt_model.transform(val_data)
    evaluator = M.evaluation.RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    valid_rmse_depth_7 = float(evaluator.evaluate(predictions))
    #print(valid_rmse_depth_7)   
    
    dt_model =  M.regression.DecisionTreeRegressor(featuresCol="features", labelCol=target, maxDepth=9)
    dt_model = dt_model.fit(new_train_data)
    predictions = dt_model.transform(val_data)
    evaluator = M.evaluation.RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    valid_rmse_depth_9 = float(evaluator.evaluate(predictions))
    #print(valid_rmse_depth_9)
    
    dt_model =  M.regression.DecisionTreeRegressor(featuresCol="features", labelCol=target, maxDepth=12)
    dt_model = dt_model.fit(new_train_data)
    predictions = dt_model.transform(val_data)
    evaluator = M.evaluation.RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    valid_rmse_depth_12 = float(evaluator.evaluate(predictions))
    #print(valid_rmse_depth_12)
    
    
    dt_model =  M.regression.DecisionTreeRegressor(featuresCol="features", labelCol=target, maxDepth=12)
    dt_model = dt_model.fit(new_train_data)
    predictions = dt_model.transform(test_data)
    evaluator = M.evaluation.RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    test_rmse = float(evaluator.evaluate(predictions))
    #print(test_rmse)
        
    
    
    
    # -------------------------------------------------------------------------
    
    
    # ---------------------- Put results in res dict --------------------------
    res = {
        'test_rmse': None,
        'valid_rmse_depth_5': None,
        'valid_rmse_depth_7': None,
        'valid_rmse_depth_9': None,
        'valid_rmse_depth_12': None,
    }
    # Modify res:
    res['test_rmse']=test_rmse
    res['valid_rmse_depth_5']=valid_rmse_depth_5
    res['valid_rmse_depth_7']=valid_rmse_depth_7
    res['valid_rmse_depth_9']=valid_rmse_depth_9
    res['valid_rmse_depth_12']=valid_rmse_depth_12

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_8')
    return res
    # -------------------------------------------------------------------------

In [24]:
res = task_8(data_io, data_dict['ml_features_train'], data_dict['ml_features_test'])
pa2.tests.test(res, 'task_8')

tests for task_8 --------------------------------------------------------------
Test 1/5 : test_rmse ... Pass
Test 2/5 : valid_rmse_depth_12 ... Pass
Test 3/5 : valid_rmse_depth_5 ... Pass
Test 4/5 : valid_rmse_depth_7 ... Pass
Test 5/5 : valid_rmse_depth_9 ... Pass
5/5 passed
-------------------------------------------------------------------------------


True

In [25]:
print ("End to end time: {}".format(time.time()-begin))

End to end time: 1082.9223971366882
